In [1]:
%config PPMagics.domain= "ccg24-hrzana-fpa-product";
%config PPMagics.autolimit=0;
%config PPMagics.fastload=True;
%config PPMagics.enable_download=True

## 1. Average sessions per user:

In [7]:
SELECT 
     case when trim(prmry_reside_cntry_code) in ('US','DE','GB') then prmry_reside_cntry_code else 'RoW' end as cntry_grp
    ,count(distinct cust_id) as total_weekly_custs
    ,count(distinct sessn_key) as total_weekly_sessions
    ,count(distinct sessn_key)/count(distinct cust_id) as avg_sessn_per_user_per_week
FROM (select
            a.cust_id
            ,a.sessn_start_dt
            ,a.sessn_key
            ,a.prmry_reside_cntry_code
      from `pypl-edw`.pp_product_analytics_datastore_views.cxp_login_active a
      where sessn_start_dt between '2025-04-13' and '2025-04-19'
      and rtrim(a.channel) = 'APP'
      and fin_acct_type_code in ('PERSONAL','PREMIER') --New segmentation
        --  and cust_acct_clsfn_key in (1,7) --older logic for consumer filter
      and lower(client_os_catg) in ('android','ios')
      group by all
     ) 
group by 1
order by 2 desc;

HTML(value='')

HTML(value='')

job id =  YXn4NyefDCE9h5FxQTXwqqIA9Bm
total bytes billed 10397679616
Total slot ms 2154083
Time executed =  13.906  seconds
.

HTML(value='')

DownloadWidget(children=(HBox(children=(Dropdown(description='File Type:', index=1, options=('tde', 'csv', 'js…

cntry_grp,total_weekly_custs,total_weekly_sessions,avg_sessn_per_user_per_week
US,14597848,47855186,3.2782356687095247
RoW,12377933,28769613,2.324266337521782
DE,11494745,26498239,2.305248093802864
GB,3014477,6790906,2.252764244013141


## 2. Repeated login sessions per user:

In [10]:
select 
        cntry_grp
        ,count(distinct cust_id) as total_weekly_custs
        ,count(distinct case when sessions_per_user>=2 then cust_id else null end) as  weekly_repeated_login_custs
        ,count(distinct case when sessions_per_user>=2 then cust_id else null end)/count(distinct cust_id) as  repeated_login_percentage
        
from (
        SELECT 
            case when trim(prmry_reside_cntry_code) in ('US','DE','GB') then prmry_reside_cntry_code else 'RoW' end as cntry_grp
            ,cust_id
            ,count(distinct sessn_key) as sessions_per_user
    FROM (select
            a.cust_id
            ,a.sessn_start_dt
            ,a.sessn_key
            ,a.prmry_reside_cntry_code
          from `pypl-edw`.pp_product_analytics_datastore_views.cxp_login_active a
          where sessn_start_dt between '2025-04-13' and '2025-04-19'
          and rtrim(a.channel) = 'APP'
          and fin_acct_type_code in ('PERSONAL','PREMIER') --New segmentation
            --  and cust_acct_clsfn_key in (1,7) --older logic for consumer filter
          and lower(client_os_catg) in ('android','ios')
          group by all
         ) 
    group by 1,2
)
group by 1;

HTML(value='')

HTML(value='')

job id =  4Dqx44jkgY7YZa8FINbuqGpc293
total bytes billed 10397679616
Total slot ms 1888556
Time executed =  16.412  seconds
.

HTML(value='')

DownloadWidget(children=(HBox(children=(Dropdown(description='File Type:', index=1, options=('tde', 'csv', 'js…

cntry_grp,total_weekly_custs,weekly_repeated_login_custs,repeated_login_percentage
US,14597848,7421275,0.508381440880875
RoW,12377933,5340654,0.4314657382617922
DE,11494745,5919424,0.5149678396519453
GB,3014477,1319204,0.43762284469246243


## 3. Weekly App login users - Transacting with Product (such as Checkout) same week

In [23]:
WITH revenue_metric AS
(
    select
         case when trim(prmry_reside_cntry_code) in ('US','DE','GB') then prmry_reside_cntry_code else 'RoW' end as cntry_grp
        ,a.cust_id
        ,max(case when b.cust_id is not null then 1 else 0 end) as user_transacted_flag
        ,sum(txn_cnt) as num_txns
        ,sum(tpv) as tpv
        ,sum(revenue) sum_rev
    from (select
            a.prmry_reside_cntry_code,
             a.cust_id
             ,last_day(a.sessn_start_dt,week) as app_login_week
          from `pypl-edw`.pp_product_analytics_datastore_views.cxp_login_active a
          where sessn_start_dt between '2025-04-13' and '2025-04-19'
          and rtrim(a.channel) = 'APP'
          and fin_acct_type_code in ('PERSONAL','PREMIER') --New segmentation
            --  and cust_acct_clsfn_key in (1,7) --older logic for consumer filter
          and lower(client_os_catg) in ('android','ios')
          group by all
         )  a
    left join (select *
                from pypl-edw.pp_peak_views.digitalwallet_daily_custlist as a
                where a.cal_dt between '2025-04-13' and '2025-04-19'
                and a.dw_product  in ('CHECKOUT')
                )b 
    on a.cust_id = b.cust_id 
    and a.app_login_week = last_day(b.cal_dt,week)
    group by 1,2
)

SELECT 
        cntry_grp,
        count(distinct cust_id) as total_app_login_custs, 
        count(distinct case when user_transacted_flag=1 then cust_id end) as XO_actives,
        count(distinct case when user_transacted_flag=1 then cust_id end)/count(distinct cust_id) as percentage_XO_transacted,
        sum(tpv) as XO_trans_processing_volume,
        sum(num_txns) as XO_total_transactions,
        sum(sum_rev) as XO_total_revenue,
        avg(coalesce(num_txns,0)) as XO_avg_TPU,
        avg(coalesce(sum_rev,0)) as XO_avg_ARPU
FROM revenue_metric
group by 1
order by 1;

HTML(value='')

HTML(value='')

job id =  A2bJYBAhbGRDEhKmYQoNqAYi1av
total bytes billed 64432898048
Total slot ms 3573330
Time executed =  25.37  seconds
.

HTML(value='')

DownloadWidget(children=(HBox(children=(Dropdown(description='File Type:', index=1, options=('tde', 'csv', 'js…

cntry_grp,total_app_login_custs,XO_actives,percentage_XO_transacted,XO_trans_processing_volume,XO_total_transactions,XO_total_revenue,XO_avg_TPU,XO_avg_ARPU
DE,11494745,8260426,0.7186262940152217,1159882784.59,21992587,24230665.75,1.9132731522099877,2.10797766718617942372797308683228727562
GB,3014477,1625812,0.53933468392693,221964381.92,4200536,5163224.75,1.393454320600224,1.71280946910525441063242479541227217856
RoW,12377933,4584951,0.3704132992156283,636145898.15,11003654,16164370.01,0.8889734659252062,1.30590220596605265192500234085933410691
US,14597848,6210337,0.42542825490442154,1064963771.06,18435208,25843795.2,1.2628716232694024,1.77038390864187652864997635267883320884


## 4. Weekly App login users - Transacting with overall DW Products - same week

In [21]:
WITH revenue_metric AS
(
    select
         case when trim(prmry_reside_cntry_code) in ('US','DE','GB') then prmry_reside_cntry_code else 'RoW' end as cntry_grp
        ,a.cust_id
        ,max(case when b.cust_id is not null then 1 else 0 end) as user_transacted_flag
        ,sum(txn_cnt) as num_txns
        ,sum(tpv) as tpv
        ,sum(revenue) sum_rev
    from (select
             a.prmry_reside_cntry_code,
             a.cust_id
             ,last_day(a.sessn_start_dt,week) as app_login_week
          from `pypl-edw`.pp_product_analytics_datastore_views.cxp_login_active a
          where sessn_start_dt between '2025-04-13' and '2025-04-19'
          and rtrim(a.channel) = 'APP'
          and fin_acct_type_code in ('PERSONAL','PREMIER') --New segmentation
            --  and cust_acct_clsfn_key in (1,7) --older logic for consumer filter
          and lower(client_os_catg) in ('android','ios')
          group by all
         )  a
    left join (select *
                from pypl-edw.pp_peak_views.digitalwallet_daily_custlist as a
                where a.cal_dt between '2025-04-13' and '2025-04-19'
                and a.dw_product not in ('DONATE - PAYPAL', 'P2P F&F', 'P2P G&S', 'DONATE', 'DONATE - GENEROSITY NETWORK', 'QRC - Instore', 'QRC - Casual','CHECKOUT',
                       'SAVINGS - FUNDS IN', 'SAVINGS - FUNDS OUT','PAYPAL CB CREDIT CARD', 'PAYPAL CREDIT', 'BNPL','ADD FUNDS','ADD FUNDS - INSTANT','XOOM', 'CHECKOUT', 
                       'P2P - Receivers', 'P2P - Senders', 'P2P F&F - Receivers', 'P2P F&F - Senders', 'P2P G&S - Receivers', 'P2P G&S - Senders')
                )b 
    on a.cust_id = b.cust_id 
    and a.app_login_week = last_day(b.cal_dt,week)
    group by 1,2
)

SELECT 
        cntry_grp,
        count(distinct cust_id) as total_app_login_custs, 
        count(distinct case when user_transacted_flag=1 then cust_id end) as DW_actives,
        count(distinct case when user_transacted_flag=1 then cust_id end)/count(distinct cust_id) as percentage_DW_transacted,
        sum(tpv) as DW_trans_processing_volume,
        sum(num_txns) as DW_total_transactions,
        sum(sum_rev) as DW_total_revenue,
        avg(coalesce(num_txns,0)) as DW_avg_TPU,
        avg(coalesce(sum_rev,0)) as DW_avg_ARPU
FROM revenue_metric
group by 1
order by 1;

HTML(value='')

HTML(value='')

job id =  byR1RCTO6pCJH0lRQ4K1A5tjQ14
total bytes billed 64432898048
Total slot ms 3282686
Time executed =  19.468  seconds
.

HTML(value='')

DownloadWidget(children=(HBox(children=(Dropdown(description='File Type:', index=1, options=('tde', 'csv', 'js…

cntry_grp,total_app_login_custs,DW_actives,percentage_DW_transacted,DW_trans_processing_volume,DW_total_transactions,DW_total_revenue,DW_avg_TPU,DW_avg_ARPU
DE,11494745,10211081,0.8883260133217397,1991617953.78,32531735,25891123.58,2.830139772565633,2.25243131361330764623312652868767423723
GB,3014477,2106712,0.6988648445484905,404767738.84,5734004,6412555.63,1.9021554982837814,2.12725312881803377501304538067465766035
RoW,12377933,6796775,0.5491042001923908,1194570782.04,15812104,24559373.67,1.277443010880735,1.98412559431368710753241272189791300373
US,14597848,9478624,0.6493165293952917,3396747113.62000000000009792167077193880686536438,39248166,48014714.16000000000000174582570622305865981602,2.6886268441759356,3.28916386579720517709197587382100588105
